In [2]:
from gazpacho import get, Soup

In [3]:
url = 'https://www.capfriendly.com/'
html = get(url)

In [4]:
soup = Soup(html)

In [5]:
table = soup.find('table', {'id': 'ich'})

In [8]:
str(table)[:100]

'<table id="ich" class="sortablex tblcf index tbl sortable"><thead><tr class="column_head"><th class='

In [9]:
trs = table.find('tr', {'class': 'tmx'})

In [10]:
tr = trs[0]

In [11]:
tr.find('a', mode='first').text

'Arizona Coyotes'

In [13]:
tr.find('td', {'data-label': 'PROJECTED CAP HIT'}, strict=True).text

'$83,657,876'

In [14]:
def parse_tr(tr):
    team = tr.find('a', mode='first').text
    cap = tr.find('td', {'data-label': 'PROJECTED CAP HIT'}, strict=True).text
    cap = float(cap.replace(',', '').replace('$', ''))
    return team, cap

In [15]:
cap_hits = [parse_tr(tr) for tr in trs]

In [16]:
url = 'https://www.hockey-reference.com/friv/playoff_prob.fcgi'
html = get(url)

In [17]:
soup = Soup(html)

In [19]:
import pandas as pd

In [20]:
east = pd.read_html(str(soup.find('table')[0]))[0]
west = pd.read_html(str(soup.find('table')[1]))[0]

In [21]:
df = pd.concat([east, west])[['Team', 'W']].reset_index(drop=True)
df['W'] = df['W'].apply(pd.to_numeric, errors='coerce')
wins = df.dropna()

In [22]:
cap_hits = pd.DataFrame(cap_hits, columns=['Team', 'spend'])
df = pd.merge(wins, cap_hits, on='Team', how='left')
df['mpw'] = round(df['spend'] / df['W'] / 1_000_000, 2)
df.sort_values('mpw', ascending=True)

,Team,W,spend,mpw
16,Colorado Avalanche,49.1,75106723.0,1.53
9,New York Islanders,48.6,75499829.0,1.55
8,Washington Capitals,50.2,81007358.0,1.61
18,Winnipeg Jets,44.9,76112344.0,1.70
0,Boston Bruins,47.5,83075527.0,1.75
11,Carolina Hurricanes,45.6,80481143.0,1.76
10,Pittsburgh Penguins,44.5,80759087.0,1.81
1,Tampa Bay Lightning,43.5,79685580.0,1.83
19,Dallas Stars,44.1,81825021.0,1.86
17,St. Louis Blues,43.9,83043070.0,1.89
